In [1]:
import pandas as pd

In [2]:
u_cols = ['user_id','age','sex','occupation','zip_code']
g_cols=['genre','genre_id']
m_cols=['movie_id','title','release_date','video_release_date','imdb_url']
users = pd.read_csv('ml-100k/u.user',sep='|',header= -1,names = u_cols, encoding='latin1') #sep 에 정규식도 들어감.
genres =  pd.read_csv('ml-100k/u.genre',names=g_cols,sep='|',encoding='latin1') 
movies = pd.read_csv('ml-100k/u.item',sep='|',names = m_cols,usecols=range(5),encoding='latin1')

In [3]:
r_cols=['user_id','movie_id','rating','unix_timestamp']
ratings=pd.read_csv('ml-100k/u.data',sep='\t',names=r_cols)
ratings['date'] = pd.to_datetime(ratings['unix_timestamp'],unit='s') #to_datetime 으로 바꿔두면 dt. 할때 aggregation 할때 좋아서 바꿔줌.
ratings.head()

,user_id,movie_id,rating,unix_timestamp,date
0,196,242,3,881250949,1997-12-04 15:55:49
1,186,302,3,891717742,1998-04-04 19:22:22
2,22,377,1,878887116,1997-11-07 07:18:36
3,244,51,2,880606923,1997-11-27 05:02:03
4,166,346,1,886397596,1998-02-02 05:33:16


read csv 와 거의 똑같은데 , 

사용자, item 과 rating 정보 세개를 통해서 추천시스템을 만들것임 - > 합쳐야함 merge로

# tidy data는 테이블마다 각각 의미가 섞이면안되며,, 분리시켜두고 실제 사용할때는 합쳐야함.  merge ,join ,concatenate 세개가 있다.

In [4]:
movie_rating = pd.merge(movies,ratings) # 알아서 조인해줌.  default 가 innerjoin 임.
lens = pd.merge(movie_rating,users)
lens.head()

,movie_id,title,release_date,video_release_date,imdb_url,user_id,rating,unix_timestamp,date,age,sex,occupation,zip_code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,308,4,887736532,1998-02-17 17:28:52,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,308,5,887737890,1998-02-17 17:51:30,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),308,4,887739608,1998-02-17 18:20:08,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,308,4,887738847,1998-02-17 18:07:27,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),308,5,887736696,1998-02-17 17:31:36,60,M,retired,95076


가장레이팅이 많은영화.

In [5]:
lens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 13 columns):
movie_id              100000 non-null int64
title                 100000 non-null object
release_date          99991 non-null object
video_release_date    0 non-null float64
imdb_url              99987 non-null object
user_id               100000 non-null int64
rating                100000 non-null int64
unix_timestamp        100000 non-null int64
date                  100000 non-null datetime64[ns]
age                   100000 non-null int64
sex                   100000 non-null object
occupation            100000 non-null object
zip_code              100000 non-null object
dtypes: datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 10.7+ MB


In [6]:
lens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 13 columns):
movie_id              100000 non-null int64
title                 100000 non-null object
release_date          99991 non-null object
video_release_date    0 non-null float64
imdb_url              99987 non-null object
user_id               100000 non-null int64
rating                100000 non-null int64
unix_timestamp        100000 non-null int64
date                  100000 non-null datetime64[ns]
age                   100000 non-null int64
sex                   100000 non-null object
occupation            100000 non-null object
zip_code              100000 non-null object
dtypes: datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 10.7+ MB


In [7]:
lens.groupby('title')

In [8]:
lens.title.value_counts().nlargest(10) # 최고상위 10 개. value_counts 는 이미 정렬되어있으니까 >> 평점이 많다는것은 인기가 있다라는것. 

Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
Name: title, dtype: int64

In [9]:
lens.title.value_counts().head(10) #영화별로몇개있는지 보는것. 583 개의 평점이 있다.  value_counts 가 카테고리별 갯수. 평점갯수가됨.

Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
Name: title, dtype: int64

# 영화당 평점 합계 , 평점 평균

In [10]:
lens.groupby('title').rating.size()

title
'Til There Was You (1997)                                     9
1-900 (1994)                                                  5
101 Dalmatians (1996)                                       109
12 Angry Men (1957)                                         125
187 (1997)                                                   41
2 Days in the Valley (1996)                                  93
20,000 Leagues Under the Sea (1954)                          72
2001: A Space Odyssey (1968)                                259
3 Ninjas: High Noon At Mega Mountain (1998)                   5
39 Steps, The (1935)                                         59
8 1/2 (1963)                                                 38
8 Heads in a Duffel Bag (1997)                                4
8 Seconds (1994)                                              4
A Chef in Love (1996)                                         8
Above the Rim (1994)                                          5
Absolute Power (1997)             

In [11]:
lens.groupby('title').rating.sum().nlargest(10)

title
Star Wars (1977)                    2541
Fargo (1996)                        2111
Return of the Jedi (1983)           2032
Contact (1997)                      1936
Raiders of the Lost Ark (1981)      1786
Godfather, The (1972)               1769
English Patient, The (1996)         1759
Toy Story (1995)                    1753
Silence of the Lambs, The (1991)    1673
Scream (1996)                       1645
Name: rating, dtype: int64

In [12]:
# user 중에서 제일 시스템에 적극적으로 사용한사람.


In [13]:
lens.groupby('user_id').size().nlargest()

user_id
405    737
655    685
13     636
450    540
276    518
dtype: int64

In [14]:
#lens.groupby(['user_id','movie_id']).size().nlargest()

In [15]:
import numpy as np

In [25]:
# mean 하고 sum 하고 동시에 쓰고싶으면 aggregate .agg 하고 인자 두개 사용가능. 
#평점수와 평균 평점을 집계한다음 평균 평점순으로 정렬
movie_stats = lens.groupby('title').agg({'rating':[np.size,np.mean]})
movie_stats.sort_values(by=[('rating','size')],ascending = False).head(10)


rating          
                                size      mean
title                                         
Star Wars (1977)                 583  4.358491
Contact (1997)                   509  3.803536
Fargo (1996)                     508  4.155512
Return of the Jedi (1983)        507  4.007890
Liar Liar (1997)                 485  3.156701
English Patient, The (1996)      481  3.656965
Scream (1996)                    478  3.441423
Toy Story (1995)                 452  3.878319
Air Force One (1997)             431  3.631090
Independence Day (ID4) (1996)    429  3.438228

In [29]:
movie_stats['rating']['size'].head()

title
'Til There Was You (1997)      9
1-900 (1994)                   5
101 Dalmatians (1996)        109
12 Angry Men (1957)          125
187 (1997)                    41
Name: size, dtype: int64

In [19]:
#평점수가 100건 이상 레이팅한 영화만 나오게함.
atleast_100 = movie_stats['rating']['size']>=100
movie_stats[atleast_100].sort_values(by=[('rating','mean')],ascending=False).head()

rating          
                                   size      mean
title                                            
Close Shave, A (1995)               112  4.491071
Schindler's List (1993)             298  4.466443
Wrong Trousers, The (1993)          118  4.466102
Casablanca (1942)                   243  4.456790
Shawshank Redemption, The (1994)    283  4.445230

------------------------------------------------------

# 추천 

### 가장 naive 한 방법 : 인기순으로 추천하기 

In [30]:
import pandas as pd
import numpy as np

In [31]:
frame = pd.read_csv('01/rating_final.csv')
cuisine = pd.read_csv('01/chefmozcuisine.csv')

In [32]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
userID            1161 non-null object
placeID           1161 non-null int64
rating            1161 non-null int64
food_rating       1161 non-null int64
service_rating    1161 non-null int64
dtypes: int64(4), object(1)
memory usage: 45.4+ KB


In [33]:
cuisine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 2 columns):
placeID     916 non-null int64
Rcuisine    916 non-null object
dtypes: int64(1), object(1)
memory usage: 14.4+ KB


In [34]:
cuisine.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


In [35]:
frame.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [44]:
frame.groupby('placeID')['rating'].count().head() 

placeID
132560     4
132561     4
132564     4
132572    15
132583     4
Name: rating, dtype: int64

In [42]:
pd.DataFrame(frame.groupby('placeID')['rating'].count()).head() # rating 을 집계

,rating
placeID,
132560,4
132561,4
132564,4
132572,15
132583,4


In [54]:
rating_count = pd.DataFrame(frame.groupby('placeID')['rating'].count()) 

In [ ]:
#pivot 은 각각의 갯수였고 pivot_table 은 각각의 aggregation  << 뭐지.?

In [53]:
frame.groupby('placeID').rating.count().head()

placeID
132560     4
132561     4
132564     4
132572    15
132583     4
Name: rating, dtype: int64

In [55]:
# merge 로 시작. 
merged = pd.merge(frame,cuisine) # merge 는 순서가 중요하다 앞뒤, inner ,outer 떄문에.

In [73]:
merged.head()

,userID,placeID,rating,food_rating,service_rating,Rcuisine
0,U1077,135085,2,2,2,Fast_Food
1,U1108,135085,1,2,1,Fast_Food
2,U1081,135085,1,2,1,Fast_Food
3,U1056,135085,2,2,2,Fast_Food
4,U1134,135085,2,1,2,Fast_Food


# 패스트 푸드에서 가장 인기가 많은 레스토랑 찾기( rating 이 많은 곳 찾기)

In [45]:
# mean 하고 sum 하고 동시에 쓰고싶으면 aggregate .agg 하고 인자 두개 사용가능. 
#평점수와 평균 평점을 집계한다음 평균 평점순으로 정렬
# movie_stats = lens.groupby('title').agg({'rating':[np.size,np.mean]})
# movie_stats.sort_values(by=[('rating','mean')],ascending = False).head(10)


rating     
                                                    size mean
title                                                        
They Made Me a Criminal (1939)                         1  5.0
Marlene Dietrich: Shadow and Light (1996)              1  5.0
Saint of Fort Washington, The (1993)                   2  5.0
Someone Else's America (1995)                          1  5.0
Star Kid (1997)                                        3  5.0
Great Day in Harlem, A (1994)                          1  5.0
Aiqing wansui (1994)                                   1  5.0
Santa with Muscles (1996)                              2  5.0
Prefontaine (1997)                                     3  5.0
Entertaining Angels: The Dorothy Day Story (1996)      1  5.0

In [85]:
merged.groupby(["Rcuisine",'placeID']).size().unstack()

placeID,132560,132572,132583,132584,132594,132608,132609,132613,132626,132630,...,135073,135074,135075,135079,135085,135086,135088,135104,135106,135109
Rcuisine,,,,,,,,,,,,,,,,,,,,,
American,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Armenian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bakery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bar_Pub_Brewery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Breakfast-Brunch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Burgers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
Cafe-Coffee_Shop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cafeteria,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN


In [76]:
merged.groupby("Rcuisine").agg({'rating':[np.size,np.mean]})
merged.sort_values(by=[('rating')],ascending = False).head(10)

,userID,placeID,rating,food_rating,service_rating,Rcuisine
0,U1077,135085,2,2,2,Fast_Food
494,U1137,135049,2,2,2,Seafood
539,U1003,132754,2,2,2,Mexican
540,U1061,132754,2,2,2,Mexican
541,U1059,132754,2,2,2,Mexican
542,U1053,135042,2,1,2,Chinese
546,U1076,135042,2,2,2,Chinese
548,U1005,135042,2,1,2,Chinese
549,U1036,135042,2,2,2,Chinese
550,U1071,135042,2,2,2,Chinese


In [94]:
# 날자 시간은 astype이 없다고 ㅎ나다.

In [95]:
frame.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [96]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
userID            1161 non-null object
placeID           1161 non-null int64
rating            1161 non-null int64
food_rating       1161 non-null int64
service_rating    1161 non-null int64
dtypes: int64(4), object(1)
memory usage: 45.4+ KB


In [97]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1042
Data columns (total 6 columns):
userID            1043 non-null object
placeID           1043 non-null int64
rating            1043 non-null int64
food_rating       1043 non-null int64
service_rating    1043 non-null int64
Rcuisine          1043 non-null object
dtypes: int64(4), object(2)
memory usage: 97.0+ KB


In [100]:
frame.food_rating.mean() # 음식 전체의 평점 평균. 

1.215331610680448

In [58]:
merged.groupby('placeID').food_rating.mean().head(10)

placeID
132560    1.000000
132572    1.000000
132583    1.000000
132584    1.500000
132594    1.200000
132608    1.166667
132609    0.600000
132613    1.333333
132626    1.000000
132630    1.166667
Name: food_rating, dtype: float64

----------

# 음식 평점이 가장 높은 지역정보 알고싶다

## 상관관계(corelation)에 따른 추천

In [59]:
geodata = pd.read_csv('01/geoplaces2.csv',encoding='latin1')

In [60]:
geodata.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [61]:
zzang = pd.merge(merged,geodata)

In [62]:
zzang.groupby('placeID').food_rating.count().nlargest()

placeID
135032    56
135052    50
135085    36
135041    34
132825    32
Name: food_rating, dtype: int64

In [63]:
zzang.head()

,userID,placeID,rating,food_rating,service_rating,Rcuisine,latitude,longitude,the_geom_meter,name,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,U1077,135085,2,2,2,Fast_Food,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,U1108,135085,1,2,1,Fast_Food,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,U1081,135085,1,2,1,Fast_Food,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,U1056,135085,2,2,2,Fast_Food,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,U1134,135085,2,1,2,Fast_Food,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none


In [64]:
zzang.placeID = zzang.name

In [65]:
zzang.head() #place ID 대신에 name을 줌.

,userID,placeID,rating,food_rating,service_rating,Rcuisine,latitude,longitude,the_geom_meter,name,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,U1077,Tortas Locas Hipocampo,2,2,2,Fast_Food,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,U1108,Tortas Locas Hipocampo,1,2,1,Fast_Food,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,U1081,Tortas Locas Hipocampo,1,2,1,Fast_Food,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,U1056,Tortas Locas Hipocampo,2,2,2,Fast_Food,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,U1134,Tortas Locas Hipocampo,2,1,2,Fast_Food,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none


# -------------------

In [66]:
#모양바꾸기.
#collaborative filtering . user item 으로 구성된 2차원테이블. 
#안에는 뭐가들어갔냐면, 삿던횟수.
#사용자하고 item 하고 , 그안에 들어갈 세가지 요소만 있으면 추천 알고리즘을 사용할 수 있다. 
#판다스에서 추천알고리즘 만드려면 표를 먼저 만들어야한다. 
#set index ,unstack , pivot, pivot_table 하는 방법이있다.
#데이터가 채워지지않는 테이블을 sparse 테이블라 한다.
place_crosstab = pd.pivot_table(data=frame,values='rating',index='userID',columns='placeID') #이렇게 pivot table 로만드는게, 추천을 위해 좋다, 추천할때는 다시 melt 해서 보여준다 이건 중간포맷
place_crosstab.head()  

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# NaN 은 0 으로 바꿨자나 , fillNa 로  NaN을 0 으로 모두 채워넣기
place_crosstab.fillna(value=0).head(10)

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
U1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
U1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1006,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1007,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
U1008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
U1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


콜레보러티브 추천 user 기반으로  유사도 상관계수 구하는게 첫번째 일.

In [68]:
place_crosstab.corr().head(10) # item 기반 collaborative 일 .

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
placeID,,,,,,,,,,,,,,,,,,,,,
132560,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132561,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132564,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132572,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.090909,NaN,NaN,-0.428571,0.645497,NaN,NaN,NaN,NaN,NaN
132583,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132584,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.000000,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.707107,NaN,NaN,NaN
132594,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.000000,NaN,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132608,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.000000,0.904534,0.816497,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132609,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.904534,1.000000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
#user 기반으로 바꾸기
place_crosstab.T.corr().head(10)

userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
userID,,,,,,,,,,,,,,,,,,,,,
U1001,1.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.301511,NaN,-1.000000,NaN,NaN,NaN,NaN
U1002,1.0,1.000000,-0.522233,NaN,NaN,NaN,NaN,NaN,-0.577350,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,1.0
U1003,NaN,-0.522233,1.000000,NaN,NaN,NaN,NaN,NaN,-0.612372,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.353553,NaN,1.0,NaN,NaN
U1004,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1005,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1006,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.534522,NaN,NaN,NaN,NaN
U1007,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1009,NaN,-0.577350,-0.612372,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,...,NaN,NaN,NaN,-0.500000,NaN,0.577350,NaN,NaN,0.612372,NaN


In [70]:
# 유사성을 판단한다. covariance, correlation 차이점 공부하기.

In [71]:
xxx =  place_crosstab.T.corr()

In [72]:
xxx['U1004'].dropna()

userID
U1004    1.000000
U1024    0.333333
U1032   -1.000000
U1045   -1.000000
U1054   -0.500000
U1071    1.000000
Name: U1004, dtype: float64

In [73]:
#1004는 1071과 제일 가깝다. 1071 이 삿던거 보고나서 똑같이 추천해준다.
#이런게 콜라보러티브 필터링. 우리는 가져다 쓰기만했을 뿐.


In [78]:
#corr()과 corrwith() 의 차이점. corr()는 칼럼들 corrwith()는두개의 데이터프레임 

similar_to_Tortas = place_crosstab.corrwith(Tortas_ratings)

corr_Tortas = pd.DataFrame(similar_to_Tortas,columns=['PearsonR'])
corr_Tortas.dropna(inplace=True)
corr_Tortas.head()

#상관간계를 이용함
#이런느낌으로 콜레버로티브 구현. 

NameError: name 'Tortas_ratings' is not defined

#KNN 을 알기위해서 scikit 을 알아야한다.


#scikit 알면 추천패키지도 알수있다.